<a href="https://colab.research.google.com/github/keyduu/03MIAR----Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Augusto_Javier_Gonz%C3%A1lez_Rodr%C3%ADguez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Augusto Javier González Rodríguez<br>
Url: https://github.com/keyduu/03MIAR----Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Augusto_Javier_Gonzalez_Rodriguez.ipynb<br>
Google Colab: https://colab.research.google.com/drive/1kpO4ObTP8-KaZMIdhBBKYsHlpDaB7k4C<br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de una jornada de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)

Problema 1. Organizar sesiones de doblaje
• Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día.

El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:

Número de actores: 10 <br>
Número de tomas : 30 <br>
Actores/Tomas : https://bit.ly/36D8IuK <br>
- 1 indica que el actor participa en la toma
- 0 en caso contrario








                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

Las soluciones se representarán en forma de lista plana, que tendrá el orden en el que se graba cada una de las escenas, al estar representado de esta manera, y tener una cantidad de 30 escenas, y poderse realizar la grabación de 6 escenas por cada día, el doblaje tendrá una duración máxima de 5 días, por lo que dividiendo la lista en grupos de 6, los seis primeros elementos serían las escenas grabadas el primer día, los seis siguientes, el segundo día, y así sucesivamente.

Para simplificar el tratamiento de los datos, se ha transformado en una tabla CSV en la que en cada fila estará una toma, y en cada columna un actor, en la tabla se marcará con 1 si el actor participa en esa escena, y un 0 en caso contrario.

La función objetivo, será la suma de la cantidad de actores que realizan grabación cada día.
$$ F=\sum_{i=1}^{D} A_i $$

La única restricción conocida es que no se pueden grabar más de 6 escenas diariamente, por lo que se incluirá un parámetro en la función de evaluación para tener en cuenta esto y poder agrupar las escenas por días.


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

El número de soluciones total es de $$ n! $$ y a cada una de las soluciones habría que aplicarle la función de valoración que tiene un número de operaciones de $$ 3n + n/k + 1 * $$ siendo n el número de escenas a grabar y k el número de actores, por lo tanto, la complejidad del problema es de $$ O(n!) $$

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Se ha elegido un algoritmo genético para resolver este problema debido a su alta complejidad. Los algoritmos deterministas no son prácticos en este caso, ya que requieren demasiados recursos computacionales. El algoritmo genético es una buena opción porque puede alcanzar un óptimo local de manera eficiente y, con suficientes generaciones, se puede acercar al óptimo global sin consumir demasiado tiempo de cálculo

In [1]:
# Importaciones de librer
import pandas as pd
import numpy as np
import random
from google.colab import drive, files

In [2]:
# Carga de datos
drive.mount('/content/drive')
files.upload()
datos = pd.read_csv('/content/Escenas.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving Escenas.csv to Escenas (5).csv


In [3]:
# Inicializar población con tomas diarias, para simplificar y hacer más sencillo el código, se realizará una lista con los indices de las escenas
# en el orden que se realizan, sin tener en cuenta que día se realizan.
def inicializar_poblacion(num_tomas, tamanyo_poblacion):
    poblacion = []
    for _ in range(tamanyo_poblacion):
        horario = list(range(num_tomas))
        random.shuffle(horario)
        poblacion.append(horario)
    return poblacion

In [4]:
# Valoración programacion
def valorar_programacion(programacion, datos, max_tomas_dia):
    total_programacion = 0

    programacion_por_dias = []
    for i in range(0, len(programacion), max_tomas_dia):
        programacion_por_dias.append(programacion[i:i + max_tomas_dia])

    for dia in programacion_por_dias:
        actores_dia_actual = np.zeros(datos.shape[1] - 1)
        valor_dia_actual = 0
        for toma in dia:
            actores_toma = datos.iloc[toma, 1:].values
            actores_dia_actual = np.clip(actores_dia_actual + actores_toma, 0, 1)
            valor_dia_actual = np.sum(actores_dia_actual)
        total_programacion = total_programacion + valor_dia_actual

    return total_programacion

In [5]:
# Cruzar
# Utilizaremos un algoritmo de cruce basado en la posicion (PBX), se toman índices aleatorios del padre1, y los indices no tomados del padre1,
# se tomarán del padre2, obteniendo aquellos valores que aún no están en el hijo, en el orden en el que aparecen en el padre2, de este modo,
# aseguramos que el hijo será una solución aceptable
def cruzar(padre1, padre2):
    # Crear una lista vacía para el hijo
    hijo = [None] * len(padre1)

    # Seleccionar posiciones aleatorias para copiar del primer padre
    num_posiciones = len(padre1) // 2
    posiciones = random.sample(range(len(padre1)), num_posiciones)

    # Copiar las posiciones seleccionadas del primer padre al hijo
    for pos in posiciones:
        hijo[pos] = padre1[pos]

    # Completar el resto del hijo con elementos del segundo padre en el orden en el que aparecen en el segundo padre
    indice_padre2 = 0
    for i in range(len(hijo)):
        if hijo[i] is None:
            while padre2[indice_padre2] in hijo:
                indice_padre2 += 1
            hijo[i] = padre2[indice_padre2]
            indice_padre2 += 1
    return hijo

In [6]:
# Mutar
# intercambia dos indices si un valor aleatorio es mayor que la tasa de mutación
def mutar(horario, tasa_mutacion):
    if random.random() < tasa_mutacion:
        indice1, indice2 = random.sample(range(len(horario)), 2)
        horario[indice1], horario[indice2] = horario[indice2], horario[indice1]

    return horario

In [7]:
# Algoritmo genético
def algoritmo_genetico(datos, max_tomas_dia=6, tamanyo_poblacion=50, generaciones=100, tasa_mutacion=0.1):
    num_tomas = datos.shape[0]
    poblacion = inicializar_poblacion(num_tomas, tamanyo_poblacion)

    for generacion in range(generaciones):
        # Ordenamos la lista de indivíduos ordenandolos de menor valor a mayor
        poblacion = sorted(poblacion, key=lambda x: valorar_programacion(x, datos, max_tomas_dia), reverse=False)

        supervivientes = poblacion[:tamanyo_poblacion // 2] # desechamos la mitad de la población con mayor valor
        nueva_poblacion = supervivientes.copy()

        while len(nueva_poblacion) < tamanyo_poblacion: # Se recupera la población hasta volver a tener la cantidad de indivíduos original
            padre1, padre2 = random.sample(supervivientes, 2) # se eligen los padres entre los supervivientes
            hijo = cruzar(padre1, padre2) #Cruce posicional
            hijo = mutar(hijo, tasa_mutacion) #Mutacion
            nueva_poblacion.append(hijo)

        poblacion = nueva_poblacion

    mejor_individuo = min(poblacion, key=lambda x: valorar_programacion(x, datos, max_tomas_dia)) # Se busca el coste mínimo

    ##### A partir de aquí, código cosmético, para dar una respuesta legible #####
    # Se pone en formato legible, con una lista de días, siendo los días la lista de indices de las escenas grabadas ese día
    mejor_individuo_dias = []
    for i in range(0, len(mejor_individuo), max_tomas_dia):
        mejor_individuo_dias.append(mejor_individuo[i:i + max_tomas_dia])
    # Se añade 1 a las escenas para tener el número de la escena (comienza por 1) en vez del índice de la escena (comienza en 0)
    mejor_programacion = []
    for dia in mejor_individuo_dias:
      mejor_programacion.append([escena + 1 for escena in dia])

    return mejor_programacion, valorar_programacion(mejor_individuo, datos, max_tomas_dia)

In [8]:
max_tomas_dia = 6
tamanyo_poblacion = 50
num_generaciones = 100
tasa_mutacion = 0.3
mejor_programacion, valor = algoritmo_genetico(datos, max_tomas_dia, tamanyo_poblacion, num_generaciones, tasa_mutacion)
print("Mejor programación: ", mejor_programacion)
print("Valoración: ", valor)

Mejor programación:  [[9, 11, 20, 6, 4, 1], [22, 18, 5, 21, 12, 8], [10, 14, 17, 29, 26, 24], [30, 2, 28, 23, 19, 27], [13, 3, 25, 15, 7, 16]]
Valoración:  29.0
